In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '4' 
print(os.environ['OMP_NUM_THREADS'])

4


In [2]:
import json
import yaml
import sys
import time
import IPython.display as ipd
import pprint
from pathlib import Path
from tqdm import tqdm

import os
import random
import numpy as np

import onnxruntime
from nix.models.TTS import NixTTSInference
from IPython.display import Audio
import soundfile as sf

Failed to create secure directory (/run/user/40760/pulse): No such file or directory
Failed to create secure directory (/run/user/40760/pulse): No such file or directory
Failed to create secure directory (/run/user/40760/pulse): No such file or directory
Failed to create secure directory (/run/user/40760/pulse): No such file or directory


In [3]:
seed = 1234
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)

In [4]:
results_base_dir = Path('./result4eval')
wav_base_dir = 'nix_wavs'
print(results_base_dir.exists())
#print(wav_base_dir.exists())

True


In [5]:
testc_json_path = Path('./ctensor4nix.json')
flag4model = 'stoch' # deter or stoch

# ./pretrained/nix-ljspeech-deterministic-v0.1
# ./pretrained/nix-ljspeech-stochastic-v0.1
if flag4model=='deter':
    model_path = Path(f'./pretrained/nix-ljspeech-deterministic-v0.1')
    results_dir_path = results_base_dir / flag4model
    wav_dir_path = results_dir_path / wav_base_dir
elif flag4model=='stoch':
    model_path = Path(f'./pretrained/nix-ljspeech-stochastic-v0.1')
    results_dir_path = results_base_dir / flag4model
    wav_dir_path = results_dir_path / wav_base_dir
else:
    print('Do not supported.')

if wav_dir_path.exists()==False:
    wav_dir_path.mkdir(parents=True)
    print(f'Make {wav_dir_path}')
else:
    print(f'Already Exists {wav_dir_path}')

if results_dir_path.exists()==False:
    results_dir_path.mkdir(parents=True)
    print(f'Make {results_dir_path}')
else:
    print(f'Already Exists {results_dir_path}')

# load ctensor4nix.json
if testc_json_path.exists()==True:
    with open(testc_json_path) as f:
        info_jsonl = [json.loads(l) for l in f.readlines()]
else:
    print('Not Exists.')

nix = NixTTSInference(model_dir = str(model_path))

Make result4eval/stoch/nix_wavs
Already Exists result4eval/stoch


## test_run

In [6]:
# Tokenize input text
#text ='Hello.'
#c, c_length, phoneme = nix.tokenize(text)
# Convert text to raw speech
index = -2
c = np.array(info_jsonl[index]['c'])
c_length = np.array(info_jsonl[index]['c_length'])
print(f'c:{c}')
print(f'c_length:{c_length}')

c:[[  0  72   0  62   0  16   0  81   0  83   0  16   0  62   0 156   0  43
    0 102   0  55   0  16   0 138   0  64   0  65   0 156   0 102   0  62
    0 131   0  16   0 156   0  43   0 102   0  16   0  72   0  55   0  16
    0 123   0 156   0  43   0 102   0 125   0 102   0 112   0  16   0  81
    0  86   0 123   0  65   0 157   0 138   0  68   0  16   0  92   0 123
    0 156   0  47   0 102   0  62   0  16   0  46   0  43   0 102   0  64
    0 156   0  87   0 158   0  61   0 177   0 125   0  51   0  16   0 138
    0  64   0  16   0  58   0 123   0 156   0  72   0  53   0  62   0 102
    0  61   0  16   0  72   0  68   0  16   0 123   0 177   0  92   0 156
    0  69   0 158   0 123   0  46   0  68   0  16   0  81   0 102   0  16
    0 156   0  43   0 135   0  85   0  68   0  16   0 138   0  64   0  16
    0  54   0 156   0  47   0 102   0  44   0  85   0   4   0]]
c_length:[213]


In [8]:
c.shape

(1, 213)

In [7]:
start_time = time.perf_counter()

xw = nix.vocalize(c, c_length)

end_time = time.perf_counter()

dt = end_time - start_time
audio_playback_time = (len(xw[0,0]) / 22050)
RTF = dt / audio_playback_time
print(f'dt[s]: {dt}')
print(f'RTF: {RTF}')
# thread1:
# dt[s]: 0.25655217794701457
# RTF: 0.1463414611892506
# thread1,['OMP_NUM_THREADS'] = '4':
#dt[s]: 0.25247742515057325
#RTF: 0.14401715709256366

2024-11-27 11:47:07.693124371 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,192,169} does not match actual shape of {1,192,213} for output 643
2024-11-27 11:47:07.725986718 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,1,169} does not match actual shape of {1,1,213} for output duration_rounded
2024-11-27 11:47:07.726120844 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,359} does not match actual shape of {1,525} for output x_masks
2024-11-27 11:47:07.825287380 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,192,359} does not match actual shape of {1,192,525} for output logsig_z
2024-11-27 11:47:07.825566251 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,192,359} does not match actual shape of {1,192,525} for output mu_z
2024-11-27 11:47:07.829000629 [W:onnxruntime:, 

dt[s]: 0.9473499958403409
RTF: 0.15542460869255592


In [10]:
# Listen to the generated speech
#wav_path = './minus1.wav'
#sf.write(wav_path, xw[0,0], samplerate=22050, subtype='PCM_16')
print((len(xw[0,0]) / 22050))
Audio(xw[0,0], rate = 22050)


6.095238095238095


## [seq] infer

In [11]:
infer_data_num: int = 101
print(f'infer_data_num: {infer_data_num}')
print(f'flag4model: {flag4model}')
print(f'model_path: {model_path}')
print(f'results_dir_path: {results_dir_path}')
print(f'wav_dir_path: {wav_dir_path}')
print(f'info_jsonl_len: {len(info_jsonl)}')

infer_data_num: 101
flag4model: stoch
model_path: pretrained/nix-ljspeech-stochastic-v0.1
results_dir_path: result4eval/stoch
wav_dir_path: result4eval/stoch/nix_wavs
info_jsonl_len: 200


In [12]:
eval_list = []

for index in tqdm(range(infer_data_num)):
    # load ctensor
    c = np.array(info_jsonl[index]['c'])
    c_length = np.array(info_jsonl[index]['c_length'])
    print(f'c:{c}')
    print(f'c_length:{c_length}')

    # inference
    start_time = time.perf_counter()

    xw = nix.vocalize(c, c_length)

    end_time = time.perf_counter()

    dt = end_time - start_time
    audio_playback_time = (len(xw[0,0]) / 22050)
    RTF4wav = dt / audio_playback_time
    print(f'dt[s]: {dt}')
    print(f'RTF: {RTF}')

    wav_path = wav_dir_path / f'{info_jsonl[index]["name"]}.wav'
    eval_dict = {
        'name': info_jsonl[index]['name'],
        'path': str(wav_path),
        'text': info_jsonl[index]['text'],
        'audio_playback_time': audio_playback_time,
        'dt': dt,
        'RTF4wav': RTF4wav,
    }
    eval_list.append(eval_dict)
    # save wav
    sf.write(wav_path, xw[0,0], samplerate=22050, subtype='PCM_16')



eval_json_path = results_dir_path / f'eval4nix_{flag4model}.json'
print(f'eval_json_path: {eval_json_path}')
if eval_json_path.exists() == False:
    with open(eval_json_path, 'w') as f:
        for entry in eval_list:
            f.write(json.dumps(entry) + '\n')
    print(f'Make {eval_json_path}')
else:
    print(f'Already Exists {eval_json_path}')
print('fin')

  0%|                                                                                                     | 0/101 [00:00<?, ?it/s]2024-11-27 11:48:27.705807484 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,192,169} does not match actual shape of {1,192,263} for output 643
2024-11-27 11:48:27.722208369 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,1,169} does not match actual shape of {1,1,263} for output duration_rounded
2024-11-27 11:48:27.722310480 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,359} does not match actual shape of {1,621} for output x_masks
2024-11-27 11:48:27.756654858 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,192,359} does not match actual shape of {1,192,621} for output logsig_z
2024-11-27 11:48:27.756800562 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape fr

c:[[  0  54   0 156   0  51   0 158   0  16   0  65   0 135   0  46   0  16
    0  61   0 156   0  47   0 102   0  16   0  81   0 157   0  70   0 125
    0 102   0  62   0  16   0  65   0 135   0  46   0  16   0  44   0  51
    0 158   0  16   0  44   0 156   0  86   0 125   0  85   0  16   0  48
    0  76   0 158   0 123   0  16   0  55   0 157   0  51   0 158   0  16
    0 102   0  48   0  16   0 156   0  43   0 102   0  16   0  65   0  86
    0  56   0  62   0  16   0  62   0  83   0  16   0 123   0 156   0 138
    0 131   0  83   0   4   0  16   0 156   0  43   0 102   0  16   0  46
    0 102   0  46   0  56   0 157   0  69   0 158   0  62   0  16   0  56
    0 156   0  57   0 135   0  16   0  65   0 156   0  43   0 102   0   4
    0  16   0 156   0  43   0 102   0  16   0  46   0 102   0  46   0  56
    0 157   0  69   0 158   0  62   0  16   0  56   0 156   0  57   0 135
    0  16   0  65   0 138   0  62   0  16   0  50   0  51   0 158   0  16
    0  50   0  72   0  46   0  16   

  1%|▉                                                                                            | 1/101 [00:01<01:49,  1.10s/it]

dt[s]: 0.9196576047688723
RTF: 0.15542460869255592
c:[[  0  81   0  72   0  62   0  16   0 156   0  69   0 158   0 123   0  61
    0  56   0 102   0  53   0  16   0  65   0 138   0  68   0  16   0  65
    0 102   0  81   0 157   0 102   0  56   0  16   0  50   0  87   0 158
    0  16   0 123   0 156   0  51   0 158   0  62   0 131   0  16   0 102
    0  56   0  81   0  83   0  16   0  50   0 156   0  43   0 135   0  61
    0   3   0]]
c_length:[93]


2024-11-27 11:48:28.784648461 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,192,169} does not match actual shape of {1,192,93} for output 643
2024-11-27 11:48:28.789706516 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,1,169} does not match actual shape of {1,1,93} for output duration_rounded
2024-11-27 11:48:28.789778038 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,359} does not match actual shape of {1,218} for output x_masks
2024-11-27 11:48:28.803210186 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,192,359} does not match actual shape of {1,192,218} for output logsig_z
2024-11-27 11:48:28.803383707 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,192,359} does not match actual shape of {1,192,218} for output mu_z
2024-11-27 11:48:28.804383235 [W:onnxruntime:, ex

...

dt[s]: 0.8711909018456936
RTF: 0.15542460869255592
c:[[  0  81   0  83   0  16   0 119   0 156   0  87   0 158   0  46   0  16
    0  58   0 157   0  69   0 158   0  61   0  83   0  44   0 156   0 102
    0  54   0 177   0 125   0  51   0   3   0  16   0  81   0  83   0  16
    0  62   0 123   0 156   0  47   0 102   0  46   0  16   0  55   0 156
    0  69   0 158   0 123   0  62   0   3   0  16   0  70   0  16   0  50
    0 156   0  72   0  56   0  46   0  61   0 138   0  55   0  16   0  56
    0 156   0  63   0 158   0  16   0  44   0 156   0 102   0  54   0  46
    0 102   0 112   0  16   0  65   0 102   0  81   0  16   0 156   0  76
    0 158   0  54   0  16   0  81   0  83   0  16   0  56   0 156   0  86
    0  61   0 177   0  61   0  85   0 123   0  51   0  16   0  48   0  83
    0  61   0 156   0 102   0  54   0 177   0 125   0  51   0  68   0   3
    0  16   0  58   0 123   0 102   0  68   0 156   0  86   0  56   0  62
    0 177   0  46   0  16   0  61   0 177   0  53   0  52  

2024-11-27 11:49:37.075958457 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,192,169} does not match actual shape of {1,192,327} for output 643
2024-11-27 11:49:37.085605911 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,1,169} does not match actual shape of {1,1,327} for output duration_rounded
2024-11-27 11:49:37.085665982 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,359} does not match actual shape of {1,893} for output x_masks
2024-11-27 11:49:37.128570869 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,192,359} does not match actual shape of {1,192,893} for output logsig_z
2024-11-27 11:49:37.128776311 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,192,359} does not match actual shape of {1,192,893} for output mu_z
2024-11-27 11:49:37.131774594 [W:onnxruntime:, 

dt[s]: 1.2832226366735995
RTF: 0.15542460869255592
c:[[  0  53   0 156   0  47   0 102   0  55   0  16   0 157   0  43   0 135
    0 125   0  83   0  64   0  16   0  81   0  83   0  16   0  53   0 156
    0  72   0  44   0 102   0  56   0   3   0  16   0  65   0 138   0  68
    0  16   0  53   0 156   0  76   0 158   0  62   0  16   0  56   0 157
    0 102   0 123   0  16   0  81   0  83   0  16   0  53   0  83   0  55
    0  58   0 156   0  72   0  56   0  51   0  83   0  56   0  16   0  44
    0  43   0 102   0  16   0  81   0  83   0  16   0 123   0 156   0  86
    0  61   0  62   0  16   0 138   0  64   0  81   0  83   0  16   0  55
    0  52   0 157   0  63   0 158   0  62   0 102   0  56   0 156   0 102
    0 123   0  68   0   3   0  16   0  72   0  56   0  46   0  16   0  58
    0 123   0 156   0  69   0 158   0  55   0  58   0  62   0  54   0  51
    0  16   0  46   0 102   0  61   0  58   0 156   0  72   0  62   0 131
    0  62   0  16   0  65   0 102   0  81   0  16   0  46  

2024-11-27 11:49:38.365370484 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,192,169} does not match actual shape of {1,192,243} for output 643
2024-11-27 11:49:38.373088345 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {-1,1,169} does not match actual shape of {1,1,243} for output duration_rounded
2024-11-27 11:49:38.373148958 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,359} does not match actual shape of {1,629} for output x_masks
2024-11-27 11:49:38.402536655 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,192,359} does not match actual shape of {1,192,629} for output logsig_z
2024-11-27 11:49:38.402700390 [W:onnxruntime:, execution_frame.cc:870 VerifyOutputSizes] Expected shape from model of {1,192,359} does not match actual shape of {1,192,629} for output mu_z
2024-11-27 11:49:38.404858764 [W:onnxruntime:, 

dt[s]: 0.888827133923769
RTF: 0.15542460869255592
eval_json_path: result4eval/stoch/eval4nix_stoch.json
Make result4eval/stoch/eval4nix_stoch.json
fin


In [13]:
eval_list[0]

{'name': 'LJ045-0049',
 'path': 'result4eval/stoch/nix_wavs/LJ045-0049.wav',
 'text': 'Lee would say that it would be better for me if I went to Russia. I did not know why. I did not know what he had in mind.',
 'audio_playback_time': 7.209795918367347,
 'dt': 0.9196576047688723,
 'RTF4wav': 0.12755667638608112}

## recheck

In [14]:
eval_jsonl_path = results_dir_path / f'eval4nix_{flag4model}.json'
print(f'eval_jsonl_path: {eval_jsonl_path}')

if eval_jsonl_path.exists() == True:
    print(f'Exist {eval_jsonl_path}')
    import json
    with open(eval_jsonl_path) as f:
        eval_jsonl_list = [json.loads(l) for l in f]
else:
    print(f'No Exists {eval_jsonl_path}')

eval_jsonl_path: result4eval/stoch/eval4nix_stoch.json
Exist result4eval/stoch/eval4nix_stoch.json


In [15]:
diff_result = list(filter(lambda x: x not in eval_list, eval_jsonl_list))
print(diff_result)

[]


## calc Indicator

In [16]:
def round_significant_digits(value, significant_digits=5):
    if value == 0:
        return 0

    import math
    scale = math.floor(-math.log10(abs(value)))  # Find the first nonzero after the decimal point
    factor = 10 ** (scale + significant_digits - 1)  # Scale to hold 5 significant digits

    rounded_value = round(value * factor,1) / factor  # Adjust and round off the scale
    return rounded_value

In [17]:
dt_list = [eval_jsonl_list[n]['dt'] for n in range(len(eval_jsonl_list))]
RTF4wav_list = [eval_jsonl_list[n]['RTF4wav'] for n in range(len(eval_jsonl_list))]
dt_nparr = np.array(dt_list[1:101])
RTF4wav_nparr = np.array(RTF4wav_list[1:101])

In [18]:
significant_digits = 5

# for culc difference time to infer text2mel
dt_mean = round_significant_digits(np.mean(dt_nparr), significant_digits=significant_digits)
dt_var = round_significant_digits(np.var(dt_nparr), significant_digits=significant_digits)
dt_std = round_significant_digits(np.std(dt_nparr), significant_digits=significant_digits)
print(f'dt ---------------------------')
print(f'dt mean: {dt_mean}')
print(f'dt var: {dt_var}')
print(f'dt std: {dt_std}')

# for culc RTF4mel to infer text2mel
RTF4wav_mean = round_significant_digits(np.mean(RTF4wav_nparr), significant_digits=significant_digits)
RTF4wav_var = round_significant_digits(np.var(RTF4wav_nparr), significant_digits=significant_digits)
RTF4wav_std = round_significant_digits(np.std(RTF4wav_nparr), significant_digits=significant_digits)
print(f' RTF ---------------------------')
print(f'RTF mean: {RTF4wav_mean}')
print(f'RTF var: {RTF4wav_var}')
print(f'RTF std: {RTF4wav_std}')

dt ---------------------------
dt mean: 0.6963699999999999
dt var: 0.057168000000000004
dt std: 0.2391
 RTF ---------------------------
RTF mean: 0.12002
RTF var: 3.7763e-06
RTF std: 0.0019433
